In [7]:
import numpy as np

class MyRegressor:
    def __init__(self) -> None:
        self.W = []
        self.X = []

    def get_parameters(self):
        return self.W
    
    def train(self, training_input_set, training_output_set):
        self.X = [[1] + line.copy() for line in training_input_set]
        XT = self.__transpose_matrix(self.X)
        P = self.__product(self.X,XT)
        P_Invers = self.__inversa(P)

        A = self.__product(P_Invers, XT)
        Y = [[el] for el in training_output_set]

        self.W = self.__product(A,Y)
        
    def __transpose_matrix(self, A):
        n, m = len(A), len(A[-1])
        XT = [[] for i in range(m)]
        for i in range(n):
            for j in range(m):
                XT[j].append(A[i][j])
        return XT

    def __product(self,A,B):
        a = np.array(A)
        b = np.array(B)
        return np.matmul(a,b)
        nr_lines = len(A)
        nr_cols = len(B[-1])
        prod = []
        for i in range(nr_lines):
            prod.append([])
            for j in range(nr_cols):
                prod[i].append(sum([A[i][k] * B[k][j] for k in range(len(A[i]))]))

        return prod          

    def __elimLinCol(self, A, i:int, j:int):
        B  = [ A[k].copy() for k in range(len(A)) if k != i ]
        for line in B:
            line.pop(j)
        return B

    def __determinant(self, A):
        arr = np.array(A)
        return np.linalg.det(arr)
        if len(A) == 1:
            return A[-1][-1]
        return sum(((-1) ** j) * A[0][j] * self.__determinant(self.__elimLinCol(A,0,j)) for j in range(len(A[0])))

    def __inversa(self, A):
        for line in A:
            print(line)
        dA = self.__determinant(A)
        print(dA)
        B = []
        nr_lines = len(A)
        nr_cols = len(A[-1])
        for i in range(nr_lines):
            B.append([])
            for j in range(nr_cols):
                B.append(self.__determinant(self.__elimLinCol(A,i,j))/dA)
        
        return B



In [8]:
regresor = MyRegressor()

In [9]:
import pandas as pd
import numpy as np

In [10]:
def getTrainingAndValidationSets(df_world_happiness):
    dataSize = df_world_happiness.shape[0]
    trainingIndexSet = np.random.choice(range(dataSize),size=int(0.8 * dataSize),replace=False)
    validationIndexSet = [i for i in range(dataSize) if i not in trainingIndexSet] 

    trainingInputSet = [df_world_happiness["Economy..GDP.per.Capita."].iloc[index] for index in trainingIndexSet ]
    trainingOutputSet = [df_world_happiness["Happiness.Score"].iloc[index] for index in trainingIndexSet ]

    validationInputSet = [df_world_happiness["Economy..GDP.per.Capita."].iloc[index] for index in validationIndexSet ]
    validationOutputSet = [df_world_happiness["Happiness.Score"].iloc[index] for index in validationIndexSet ]
    
    return trainingInputSet, trainingOutputSet, validationInputSet, validationOutputSet

In [11]:
world_happiness = pd.read_csv("datas/v1_world-happiness-report-2017.csv",delimiter=',', header='infer')

In [12]:
training_input_set, training_output_set, validation_input_set, validation_output_set = getTrainingAndValidationSets(world_happiness)

training_input_set = [[el] for el in training_input_set]
regresor.train(training_input_set, training_output_set)

[1.02621981 1.0976489  1.13888323 1.21751103 1.21717522 1.16025779
 1.21296019 1.26441637 1.22416908 1.01483601 1.1827216  1.24036438
 1.10500166 1.20908048 1.18153919 1.17968956 1.00366651 1.23375086
 1.11770086 1.24866981 1.27402262 1.25340983 1.18672967 1.2080022
 1.17680051 1.09137533 1.06980365 1.18169547 1.         1.17840983
 1.09499883 1.1831014  1.07710568 1.18732706 1.06085907 1.1470242
 1.24003536 1.12768583 1.20880069 1.0709254  1.06500933 1.19402059
 1.0594445  1.23702318 1.01927588 1.09654313 1.19796282 1.23952066
 1.06176329 1.07769504 1.22589662 1.14486681 1.24197915 1.19403097
 1.17506813 1.17238213 1.21956072 1.05671074 1.08276585 1.25125546
 1.13500631 1.11938742 1.20823791 1.08436654 1.20285756 1.30292435
 1.22943446 1.09744101 1.04951817 1.10679246 1.09075582 1.16130884
 1.12828069 1.14582942 1.05965668 1.16762919 1.11597889 1.20755234
 1.04945924 1.03959883 1.22270925 1.21903354 1.14218899 1.1731496
 1.16047001 1.23214045 1.1309918  1.05493055 1.20414714 1.1080406

AttributeError: 'numpy.ndarray' object has no attribute 'pop'